In [ ]:
from langchain_openai import ChatOpenAI
from langchain_cohere import ChatCohere
from dotenv import load_dotenv
from os import getenv

load_dotenv()

API_KEY = getenv("OPENAI_API_KEY")
MODEL_NAME_1 = 'openai/gpt-4o-mini'
MODEL_NAME_2 = 'anthropic/claude-sonnet-4'
MODEL_NAME_3 = 'google/gemini-2.0-flash-001'


# Initialize the LLM
agent_1_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)

agent_2_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_2,
)

validator_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)

In [ ]:
agent1_memory = []
agent2_memory = []

sentiment_array_1 = []
sentiment_array_2 = []

In [ ]:
# Updated LLM vs LLM Prisoner's Dilemma with scenario-based prompt adaptation
import random
from textblob import TextBlob
from typing import List, Tuple, TypedDict, Dict

from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
import time

# === Define the Game State ===
class GameState(TypedDict):
    history: List[Tuple[str, str]]           # (agent1_move, agent2_move)
    scores: Tuple[int, int]                  # (agent1_score, agent2_score)
    round: int
    max_rounds: int
    
    current_move_1: str
    current_move_2: str
    validated_move_1: str
    validated_move_2: str
    
    scenario: Dict[str, any]                 # Scenario metadata

# === Scenario Example Structure ===
default_scenario = {
    "agent1_name": "Agent 1",
    "agent2_name": "Agent 2",
    "action_labels": ["Cooperate", "Defect"],
    "payoff": {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    }
}

# === Define Move Nodes ===
def agent1_move(state: GameState) -> GameState:
    a1 = state["scenario"].get("agent1_name", "Agent 1")
    a2 = state["scenario"].get("agent2_name", "Agent 2")
    labels = state["scenario"].get("action_labels", ["Cooperate", "Defect"])

    if not state["history"]:
        prompt = (
            f"This is the first round.\n"
            f"Choose now: '{labels[0]}' or '{labels[1]}'?"
        )
    else:
        last_moves = state["history"][-1]
        
        if state["round"] == state["max_rounds"] - 1:
            prompt = (
                f"This is the last round.\n"
                f"Previously, you ({a1}) chose to '{last_moves[0]}', and {a2} chose to '{last_moves[1]}'.\n"
                f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
            )
        else:
            prompt = (
                f"Previously, you ({a1}) chose to '{last_moves[0]}', and {a2} chose to '{last_moves[1]}'.\n"
                f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
            )

    agent1_memory.append(HumanMessage(content=prompt))
    move = agent_1_llm.invoke(agent1_memory).content.strip()
    agent1_memory.append(AIMessage(content=move))
    
    return {**state, "current_move_1": move}

def agent2_move(state: GameState) -> GameState:
    a1 = state["scenario"].get("agent1_name", "Agent 1")
    a2 = state["scenario"].get("agent2_name", "Agent 2")
    labels = state["scenario"].get("action_labels", ["Cooperate", "Defect"])

    if not state["history"]:
        prompt = (
            f"This is the first round.\n"
            f"Choose now: '{labels[0]}' or '{labels[1]}'?"
        )
    else:
        last_moves = state["history"][-1]
        
        if state["round"] == state["max_rounds"] - 1:
            prompt = (
                f"This is the last round.\n"
                f"Previously, you ({a2}) chose to '{last_moves[1]}', and {a1} chose to '{last_moves[0]}'.\n"
                f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
            )
        else:
            prompt = (
                f"Previously, you ({a2}) chose to '{last_moves[1]}', and {a1} chose to '{last_moves[0]}'.\n"
                f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
            )

    agent2_memory.append(HumanMessage(content=prompt))
    move = agent_2_llm.invoke(agent2_memory).content.strip()
    agent2_memory.append(AIMessage(content=move))
    
    return {**state, "current_move_2": move}

# === Parse and Validate Moves ===
def parse_moves(state: GameState) -> GameState:
    move1 = state["current_move_1"]
    move2 = state["current_move_2"]
    labels = state["scenario"].get("action_labels", ["Cooperate", "Defect"])
    a1 = state["scenario"].get("agent1_name", "Agent 1")
    a2 = state["scenario"].get("agent2_name", "Agent 2")

    # Standardized validator prompt
    validator_prompt = (
        f"You are a strict validator in a repeated dilemma game.\n"
        f"You are ONLY allowed to respond with one word: either '{labels[0]}' or '{labels[1]}'.\n"
        f"Do NOT include any explanation, punctuation, or additional words.\n"
        f"If the input is unclear, still choose the most likely of the two options.\n"
        f"Your response MUST be exactly '{labels[0]}' or '{labels[1]}', with no variation."
    )
    
    # Validate Agent 1's move
    prompt_agent1 = (
        f"{a1} said: {move1}\n\n"
        f"Based on this message, what is their most likely final choice?\n"
        f"Respond with EXACTLY ONE WORD — either '{labels[0]}' or '{labels[1]}'.\n"
        f"DO NOT say anything else."
    )
    
    agent1_memory_validator = [
        SystemMessage(content=validator_prompt),
        HumanMessage(content=prompt_agent1)
    ]
    
    # Validate Agent 2's move
    prompt_agent2 = (
        f"{a2} said: {move2}\n\n"
        f"Based on this message, what is their most likely final choice?\n"
        f"Respond with EXACTLY ONE WORD — either '{labels[0]}' or '{labels[1]}'.\n"
        f"DO NOT say anything else."
    )
    
    agent2_memory_validator = [
        SystemMessage(content=validator_prompt),
        HumanMessage(content=prompt_agent2)
    ]
    
    # Get validated moves
    validated_move_1 = validator_llm.invoke(agent1_memory_validator).content.strip()
    validated_move_2 = validator_llm.invoke(agent2_memory_validator).content.strip()
    
    print(f"\n=== Agent Reasoning ===")
    print(f"{a1}: {move1}")
    print(f"Validated Response for {a1}: {validated_move_1}\n")
    print(f"{a2}: {move2}")
    print(f"Validated Response for {a2}: {validated_move_2}")

    return {
        **state, 
        "validated_move_1": validated_move_1,
        "validated_move_2": validated_move_2
    }

def validate_moves(state: GameState) -> str:
    """Check if both moves are valid, if not, restart the round"""
    valid_moves = state["scenario"].get("action_labels", ["Cooperate", "Defect"])
    
    if (state["validated_move_1"] not in valid_moves or 
        state["validated_move_2"] not in valid_moves):
        
        # Remove last messages from agent memories if validation failed
        if len(agent1_memory) >= 2:
            agent1_memory.pop()
            agent1_memory.pop()
        if len(agent2_memory) >= 2:
            agent2_memory.pop()
            agent2_memory.pop()
        
        return "agent1_move"
    else:
        return "sentiment_analysis"

def sentiment_analysis(state: GameState) -> GameState:
    """Analyze sentiment of both agents' responses"""
    text1 = state["current_move_1"]
    text2 = state["current_move_2"]
    
    blob1 = TextBlob(text1)
    blob2 = TextBlob(text2)
    
    # Store sentiment analysis (you can modify this based on your needs)
    sentiment_array_1.append(blob1.sentiment)
    sentiment_array_2.append(blob2.sentiment)
    
    return {**state}

def score_round(state: GameState) -> GameState:
    """Score the round based on validated moves"""
    move1 = state["validated_move_1"]
    move2 = state["validated_move_2"]
    payoff = state["scenario"].get("payoff", default_scenario["payoff"])

    score1, score2 = payoff.get((move1, move2), (0, 0))
    new_scores = (state["scores"][0] + score1, state["scores"][1] + score2)
    new_history = state["history"] + [(move1, move2)]
    new_round = state["round"] + 1

    # Print round summary
    print(f"\n=== Round {new_round} ===")
    print(f"{state['scenario'].get('agent1_name', 'Agent 1')}: {move1}")
    print(f"{state['scenario'].get('agent2_name', 'Agent 2')}: {move2}")
    print(f"Scores: {new_scores[0]} vs {new_scores[1]}")
    print("-" * 40)

    return {
        **state,
        "scores": new_scores,
        "history": new_history,
        "round": new_round
    }

def check_game_over(state: GameState) -> str:
    """Check if the game should end"""
    print(f"Checking round {state['round']} / {state['max_rounds']}\n")
    time.sleep(1)
    return END if state["round"] >= state["max_rounds"] else "agent1_move"

# === Build the Graph ===
graph = StateGraph(GameState)

graph.add_node("agent1_move", agent1_move)
graph.add_node("agent2_move", agent2_move)
graph.add_node("parse_moves", parse_moves)
graph.add_node("sentiment_analysis", sentiment_analysis)
graph.add_node("score_round", score_round)

graph.set_entry_point("agent1_move")
graph.add_edge("agent1_move", "agent2_move")
graph.add_edge("agent2_move", "parse_moves")
graph.add_conditional_edges("parse_moves", validate_moves)
graph.add_edge("sentiment_analysis", "score_round")
graph.add_conditional_edges("score_round", check_game_over)

graph = graph.compile()

# === Initialize required variables (you'll need to define these) ===
# agent1_memory = []
# agent2_memory = []
# sentiment_array_1 = []
# sentiment_array_2 = []
# agent_1_llm = your_llm_instance
# agent_2_llm = your_llm_instance  
# validator_llm = your_llm_instance

In [13]:
import os
import json
from datetime import datetime
from langchain.schema import HumanMessage, AIMessage, SystemMessage

def extract_model_name(llm):
    return getattr(llm, "model_name", "unknown_model").replace("/", "_").replace(":", "_")

def get_timestamp():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def ensure_timestamped_folder(timestamp: str):
    path = os.path.join("./results/base", timestamp)
    os.makedirs(path, exist_ok=True)
    return path

def save_game_state(state, agent_1_llm, agent_2_llm, timestamp: str, reward_matrix: dict):
    folder = ensure_timestamped_folder(timestamp)
    agent_1_model_name = extract_model_name(agent_1_llm)
    agent_2_model_name = extract_model_name(agent_2_llm)
    
    filename = os.path.join(folder, f"final_game_state_.json")

    round_breakdown = []
    cumulative_scores = [0, 0]

    for round_num, (move1, move2) in enumerate(state["history"], start=1):
        reward = reward_matrix.get((move1, move2), (0, 0))
        cumulative_scores[0] += reward[0]
        cumulative_scores[1] += reward[1]
        round_breakdown.append({
            "round": round_num,
            "agent1_move": move1,
            "agent2_move": move2,
            "reward": reward,
            "cumulative_scores": tuple(cumulative_scores),
        })

    full_state = {
        "summary": {
            "agent_1_model" : agent_1_model_name,
            "agent_2_model" : agent_2_model_name,
            "total_score_agent1": cumulative_scores[0],
            "total_score_agent2": cumulative_scores[1],
            "final_round": state["round"],
            "max_rounds": state["max_rounds"]
        },
        "final_moves": {
            "agent1": state["current_move_1"],
            "agent2": state["current_move_2"]
        },
        "rounds": round_breakdown
    }

    with open(filename, 'w') as f:
        json.dump(full_state, f, indent=2)

    return filename


def save_conversation(messages, agent_label: str, agent_llm, timestamp: str):
    folder = ensure_timestamped_folder(timestamp)
    model_name = extract_model_name(agent_llm)
    filename = os.path.join(folder, f"dialogue_{model_name}_{agent_label}.json")

    labelled_messages = []
    for msg in messages:
        if isinstance(msg, SystemMessage):
            labelled_messages.append({"role": "system", "content": msg.content})
        elif isinstance(msg, HumanMessage):
            labelled_messages.append({"role": "human", "content": msg.content})
        elif isinstance(msg, AIMessage):
            labelled_messages.append({"role": "ai", "content": msg.content})

    with open(filename, 'w') as f:
        json.dump(labelled_messages, f, indent=2)

    return filename

In [ ]:
# === Run the Game ===

# Initialize the LLM
agent_1_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)

agent_2_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_2,
)

validator_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)

base_scenario = {
    "agent1_name": "Agent 1",
    "agent2_name": "Agent 2",
    "action_labels": ["Cooperate", "Defect"],
    "payoff": {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    }
}

prompt = (
    "You are Agent 1. You and Agent 2 will interact over multiple rounds. "
    "In each round, You and Agent 2 must choose one of two actions: 'Cooperate' or 'Defect'.\n\n"
    "Rules:\n"
    " - If the both of you choose to Cooperate, you will each receives 3 points.\n"
    " - If you choose to Cooperate and Agent 2 chooses to Defect, you will receive 0 points and Agent 2 will receive 5 points.\n"
    " - If you choose to Defect and Agent 2 chooses to Cooperates, you will receive 5 points and Agent 2 will receive 0 points.\n"
    " - If both of you choose to Defect, you will each receive 1 point.\n\n"
)

prompt1 = (
    "You are Agent 2. You and Agent 1 will interact over multiple rounds. "
    "In each round, You and Agent 1 must choose one of two actions: 'Cooperate' or 'Defect'.\n\n"
    "Rules:\n"
    " - If the both of you choose to Cooperate, you will each receives 3 points.\n"
    " - If you choose to Cooperate and Agent 1 chooses to Defect, you will receive 0 points and Agent 1 will receive 5 points.\n"
    " - If you choose to Defect and Agent 1 chooses to Cooperates, you will receive 5 points and Agent 1 will receive 0 points.\n"
    " - If both of you choose to Defect, you will each receive 1 point.\n\n"
)


sentiment_array_1 = []
sentiment_array_2 = []

agent1_memory = [
    SystemMessage(content=prompt)
]

agent2_memory = [
    SystemMessage(content=prompt1)
]


initial_state = {
    "history": [],
    "scores": (0, 0),
    "round": 0,
    "max_rounds": 3,
    "current_move_1": "",
    "current_move_2": "",
    "validated_move_1": "",
    "validated_move_2": "",
    "scenario": base_scenario
}

final_state = graph.invoke(
    initial_state,
    config={"recursion_limit": 1000}
)

# === Final Summary ===
print("\n=== Final Game Summary ===")
for idx, (m1, m2) in enumerate(final_state["history"], 1):
    print(f"Round {idx}: Agent 1 -> {m1}, Agent 2 -> {m2}")

print(f"\nFinal Score:")
print(f"Agent 1: {final_state['scores'][0]}")
print(f"Agent 2: {final_state['scores'][1]}")

if final_state['scores'][0] > final_state['scores'][1]:
    print("Winner: Agent 1!")
elif final_state['scores'][0] < final_state['scores'][1]:
    print("Winner: Agent 2!")
else:
    print("It's a tie!")

In [ ]:
reward_matrix = {
    ("Cooperate", "Cooperate"): (3, 3),
    ("Cooperate", "Defect"): (0, 5),
    ("Defect", "Cooperate"): (5, 0),
    ("Defect", "Defect"): (1, 1)
}

timestamp = get_timestamp()
save_conversation(agent1_memory, "agent_1", agent_1_llm, timestamp)
save_conversation(agent2_memory, "agent_2", agent_2_llm, timestamp)
save_game_state(final_state, agent_1_llm, agent_2_llm, timestamp, reward_matrix)

In [ ]:
# After running the game
saved_files = save_complete_game_data(
    final_state, agent1_memory, agent2_memory, 
    sentiment_array_1, sentiment_array_2,
    agent_1_llm, agent_2_llm
)

# Generate all plots
plot_game_summary(saved_files["game_state_file"], save_plots=True)
plot_dual_sentiment_trace(
    saved_files["agent1_sentiment_file"], 
    saved_files["agent2_sentiment_file"]
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.DataFrame(final_state["history"], columns=["Agent1", "Agent2"])
df["Round"] = df.index + 1
df["Agent1_Coop"] = df["Agent1"].apply(lambda x: 1 if x == "Cooperate" else 0)
df["Agent2_Coop"] = df["Agent2"].apply(lambda x: 1 if x == "Cooperate" else 0)


plt.figure(figsize=(8, 4))
plt.plot(df["Round"], df["Agent1_Coop"], label="Agent 1", marker='o')
plt.plot(df["Round"], df["Agent2_Coop"], label="Agent 2", marker='s')
plt.title("Cooperation Over Rounds")
plt.xlabel("Round")
plt.ylabel("Cooperation (1=Cooperate, 0=Defect)")
plt.ylim(-0.1, 1.1)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


score_df = pd.DataFrame(columns=["Round", "Agent1_Score", "Agent2_Score"])
a1, a2 = 0, 0
payoff = {
    ("Cooperate", "Cooperate"): (3, 3),
    ("Cooperate", "Defect"):    (0, 5),
    ("Defect", "Cooperate"):    (5, 0),
    ("Defect", "Defect"):       (1, 1)
}
for idx, (m1, m2) in enumerate(final_state["history"], 1):
    s1, s2 = payoff[(m1, m2)]
    a1 += s1
    a2 += s2
    score_df.loc[idx-1] = [idx, a1, a2]

plt.figure(figsize=(8, 4))
plt.plot(score_df["Round"], score_df["Agent1_Score"], label="Agent 1", marker='o')
plt.plot(score_df["Round"], score_df["Agent2_Score"], label="Agent 2", marker='s')
plt.title("Cumulative Scores Over Time")
plt.xlabel("Round")
plt.ylabel("Score")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import requests
import json

API_KEY = getenv("OPENAI_API_KEY")
response = requests.get(
  url="https://openrouter.ai/api/v1/auth/key",
  headers={
    "Authorization": f"Bearer {API_KEY}"
  }
)

print(json.dumps(response.json(), indent=2))

In [ ]:
agent_1_llm

In [ ]:
import os
import json
from datetime import datetime
from langchain.schema import HumanMessage, AIMessage, SystemMessage

def extract_model_name(llm):
    return getattr(llm, "model_name", "unknown_model").replace("/", "_").replace(":", "_")

def get_timestamp():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def ensure_timestamped_folder(timestamp: str):
    path = os.path.join("./results/base", timestamp)
    os.makedirs(path, exist_ok=True)
    return path

def save_game_state(state, agent_1_llm, agent_2_llm, timestamp: str, reward_matrix: dict):
    folder = ensure_timestamped_folder(timestamp)
    agent_1_model_name = extract_model_name(agent_1_llm)
    agent_2_model_name = extract_model_name(agent_2_llm)
    
    filename = os.path.join(folder, f"final_game_state_.json")

    round_breakdown = []
    cumulative_scores = [0, 0]

    for round_num, (move1, move2) in enumerate(state["history"], start=1):
        reward = reward_matrix.get((move1, move2), (0, 0))
        cumulative_scores[0] += reward[0]
        cumulative_scores[1] += reward[1]
        round_breakdown.append({
            "round": round_num,
            "agent1_move": move1,
            "agent2_move": move2,
            "reward": reward,
            "cumulative_scores": tuple(cumulative_scores),
        })

    full_state = {
        "summary": {
            "agent_1_model" : agent_1_model_name,
            "agent_2_model" : agent_2_model_name,
            "total_score_agent1": cumulative_scores[0],
            "total_score_agent2": cumulative_scores[1],
            "final_round": state["round"],
            "max_rounds": state["max_rounds"]
        },
        "final_moves": {
            "agent1": state["current_move_1"],
            "agent2": state["current_move_2"]
        },
        "rounds": round_breakdown
    }

    with open(filename, 'w') as f:
        json.dump(full_state, f, indent=2)

    return filename


def save_conversation(messages, agent_label: str, agent_llm, timestamp: str):
    folder = ensure_timestamped_folder(timestamp)
    model_name = extract_model_name(agent_llm)
    filename = os.path.join(folder, f"dialogue_{agent_label}_{model_name}.json")

    labelled_messages = []
    for msg in messages:
        if isinstance(msg, SystemMessage):
            labelled_messages.append({"role": "system", "content": msg.content})
        elif isinstance(msg, HumanMessage):
            labelled_messages.append({"role": "human", "content": msg.content})
        elif isinstance(msg, AIMessage):
            labelled_messages.append({"role": "ai", "content": msg.content})

    with open(filename, 'w') as f:
        json.dump(labelled_messages, f, indent=2)

    return filename
